En esta libreta haremos un análisis exploratorio de los datos (EDA por sus siglas en inglés) con la intención de conocer la mayor cantidad posible de información contenida en el conjunto de datos provisto.

Un checklist básico y clasico que podriamos seguir sería:

- estadisticas descriptivas,
- nulos, vacios, tamaño, etc

In [2]:
import pandas as pd

In [4]:
calendar = pd.read_csv('../data/calendar.csv')
test_weights = pd.read_csv('../data/test_weights.csv')
inventory = pd.read_csv('../data/inventory.csv')
sales_train = pd.read_csv('../data/sales_train.csv')
sales_test = pd.read_csv('../data/sales_test.csv')
solution = pd.read_csv('../data/solution.csv')

In [5]:
calendar.head()

,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
0,2022-03-16,NaN,0,0,0,0,Frankfurt_1
1,2020-03-22,NaN,0,0,0,0,Frankfurt_1
2,2018-02-07,NaN,0,0,0,0,Frankfurt_1
3,2018-08-10,NaN,0,0,0,0,Frankfurt_1
4,2017-10-26,NaN,0,0,0,0,Prague_2
